In [ ]:
%matplotlib inline

import os
import sys
import random
import tables as tb
import numpy  as np

import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib.patches import Ellipse
from mpl_toolkits.mplot3d import Axes3D
from collections import namedtuple

import invisible_cities.reco.paolina_functions as plf
import invisible_cities.reco.peak_functions as pf
import invisible_cities.reco.xy_algorithms as xy
from   invisible_cities.reco    .dst_functions   import load_xy_corrections
from   invisible_cities.reco    .corrections     import LifetimeCorrection

from   invisible_cities.database import load_db
from   invisible_cities.core.configure         import configure
from   invisible_cities.core.exceptions import SipmEmptyList
from   invisible_cities.core.exceptions import ClusterEmptyList
from   invisible_cities.core.exceptions import SipmZeroCharge

from   invisible_cities.io.pmap_io             import load_pmaps
from   invisible_cities.reco                   import pmaps_functions  as pmp
from   invisible_cities.reco    .xy_algorithms import barycenter
from   invisible_cities.reco    .xy_algorithms import corona
from   invisible_cities.evm.event_model        import Hit, Cluster
from   invisible_cities.types.ic_types         import xy
from   invisible_cities.reco.tbl_functions     import get_event_numbers_and_timestamps_from_file_name
from   invisible_cities.core.system_of_units_c import units

from   invisible_cities.filters.s1s2_filter    import s1s2_filter
from   invisible_cities.filters.s1s2_filter    import s2si_filter
from   invisible_cities.filters.s1s2_filter    import S12Selector

Reconstruction functions.

This functionality will be included in the Penthesilea branch.

In [ ]:
# Configuration information
#tot_Emin = 250000; tot_Emax = 550000
PATH_IN  = os.path.join('/Users/paola/Software/ic_data/dst_NEXT_v1_00_05_Tl_ACTIVE_0_0_7bar_pmaps_open_1000.h5')
#cfile = '/Users/paola/NEXT/NEXT-WHITE/Topology/notebooks/s12_selector.conf'
corrections   = "/Users/paola/Software/ic_data/MCmap.h5"
XYcorrection  = load_xy_corrections(corrections)
LT, u_LT      = 1100, 50
e_lifetime = 1100.
drift_v = 1. 
LTcorrection  = LifetimeCorrection(LT, u_LT)
run_number    = -4529
nmin, nmax    =    1, 100
zmin, zmax    =    10, 522
lmin, lmax    =    0, 600
rmin, rmax    =    0, 250
tot_charge = 150
n_rebin = 1
corona_opts   = {
    "Qthr"          : 0.,
    "Qlm"           : 10.,
    "lm_radius"     : 20.,
    "new_lm_radius" : 25.,
    "msipm"         :  6,
}

# read the configuration file
#conf = configure(['bar', cfile])
#_s1s2_selector = S12Selector(**conf.as_dict)

In [ ]:
def rebin(arr, n_rebin, op=np.sum):
    n_bins  = int(np.ceil(arr.size/n_rebin))
#    return np.array([np.sum(arr[i*n_rebin:(i+1)*n_rebin]) for i in range(n_bins)])
    resized = np.resize(arr, (n_bins, n_rebin))
    return op(resized, axis=1)

In [ ]:
DataSiPM        = load_db.DataSiPM(run_number)
data_xs         = DataSiPM.X.values
data_ys         = DataSiPM.Y.values

# 
Read the PMaps and construct the hit collection for a given event

This will be performed in the future by Penthesilea.

In [ ]:
hitc_evt = []
evtnum_hitc = []
tbl_ri = 0
tot_evts_sum = 0

peak_energy = []
corr_peak_energy = []
### cath_energy_evt = [] ###


S1s, S2s, S2Sis = load_pmaps(PATH_IN)
event_numbers, timestamps = get_event_numbers_and_timestamps_from_file_name(PATH_IN)
#ftbl = tb.open_file(PATH_IN,'r')
tot_evts_sum += len(event_numbers)
#for evt_number, evt_time in zip(event_numbers,timestamps):
for evt_number in event_numbers:
    hitc = []

    s1   = S1s  .get(evt_number, {})
    s2   = S2s  .get(evt_number, {})
    s2si = S2Sis.get(evt_number, {})

    if(not s1 or not s2 or not s2si): continue
    if(len(s2.s2d) != len(s2si.s2sid)): continue
    if (len(s1.s1d) != 1): continue
    t, e = next(iter(s1.s1d.values()))
    S1t  = t[np.argmax(e)]
    #S2, Si = rebin_s2(S2, Si)

    npeak = 0            
    for peak_no, (t, E) in sorted(s2.s2d.items()):
    #    print("Tot energy of peak {} = {}".format(peak_no, np.sum(E)))
        peak_energy.append(np.sum(E))
        #Container for cathod energies for slices with hits
        cath_energy = []
        
        si = s2si.s2sid[peak_no]
        nslices = int(np.ceil(t.size/n_rebin))
        
        t = rebin(t, n_rebin, op=np.mean)
        E = rebin(E, n_rebin)
        
        Q = {sipm: rebin(q, n_rebin) for sipm, q in si.items()}
        Q = [{sipm:(qs[i] if i < len(qs) else 0) for sipm, qs in Q.items()} for i in range(nslices)]
        e_left = 0
 #       for slice_no, (t_slice, e_slice) in enumerate(zip(t_peak, e_peak)):
  #      print("Number of slices = {}".format(nslices) )
        for slice_no in range(nslices):
  #          print("Slice no {}".format(slice_no))
            hits_slice = []
            sipms, qsipm = list(map(list, zip(*Q[slice_no].items())))
            if np.sum(qsipm) < tot_charge:
  #              print("Too low charge in SiPMs: {} energy goes to next slice.".format(E[slice_no]))
                e_left += E[slice_no]
            else:
                xsipm = data_xs[sipms]
                ysipm = data_ys[sipms]
                possipm  = list(zip(xsipm, ysipm))
                
                try:
                    clusters = corona(np.array(possipm), np.array(qsipm), **corona_opts)                  
                except ClusterEmptyList:
                    clusters = barycenter(possipm, qsipm)
    #            print("Raw energy in slice {} = {}.".format(slice_no, E[slice_no]+ e_left))

                drift_time      = (t[slice_no] - S1t) / 1000.
                z               = dft * drift_v 
                e_corrLT        = (E[slice_no] + e_left) * np.exp(drift_time/e_lifetime) 
                
                qsum = sum(cls.Q for cls in clusters)
                for cluster in clusters:
                    c_energy = e_corrLT * cluster.Q / qsum * XYcorrection(cluster.X, cluster.Y).value[0]
                    hit      = Hit(peak_no, cluster, z, c_energy)
                    #print("hit at ({0},{1},{2};{3})".format(hit.X,hit.Y,hit.Z,hit.E))
                    #hitc.append(hit)
                    hits_slice.append(hit)
                cath_energy.append(E[slice_no] + e_left)
                e_left = 0
                hitc.append(hits_slice)
        if(len(cath_energy) > 0 and e_left > 0):
         #   print("We have some energy left: {}".format(e_left))
        #    hitc[-1].Q += e_left
            cath_energy_last = cath_energy[-1]
            cath_energy_last += e_left
            cath_energy[-1] = cath_energy_last

            ## redistribution of energy in the last slice with hits
            last_slice = hitc[-1]
            qsum_last = sum(h.Q for h in last_slice)
            for hit in last_slice:
                e_corrLT_last = cath_energy_last * np.exp(hit.Z/drift_v/e_lifetime)
                hit.energy = e_corrLT_last * hit.Q / qsum_last * XYcorrection(hit.X, hit.Y).value[0]
 #           npeak += 1
        corr_e_peak = 0.
        for slc in hitc:
            for hit in slc:
                corr_e_peak += hit.energy
        if corr_e_peak == 0.:
            print(evt_number)
        corr_peak_energy.append(corr_e_peak)
        hitc_evt.append(hitc)
###        cath_energy_evt.append(cath_energy) ###



        #    evtnum_hitc.append(evt_number)
#            print("***********************************")
            
            # record MC information if available
#            if(ftbl.__contains__('/MC')):
#                hitcm = []
#                mctbl = ftbl.root.MC.MCTracks
#                while(tbl_ri < len(mctbl) and mctbl[tbl_ri]['event_indx'] != evt_number):
#                    tbl_ri += 1
#                while(tbl_ri < len(mctbl) and mctbl[tbl_ri]['event_indx'] == evt_number):
#                    hpos = mctbl[tbl_ri]['hit_position']
#                    henergy = mctbl[tbl_ri]['hit_energy']
#                    hit = Hit(hpos[0],hpos[1],hpos[2],henergy)
#                    hitcm.append(hit)
#                    tbl_ri += 1
#                hitc_mc.append(hitcm)

#ftbl.close()
#print("Produced hit collections for {0} events ({1} events in MC truth).".format(len(hitc_evt),len(hitc_mc)))

In [ ]:
fig = plt.figure(1)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(corr_peak_energy,bins=4000,label='Corrected energy')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([100000,900000])
plt.xlim([0,2000])
plt.xlabel('Energy (pes)')
plt.ylabel('Counts/bin')

In [ ]:
print(peak_energy)

In [ ]:
ax2 = fig.add_subplot(121);
plt.hist(peak_energy,bins=40,label='Raw energy')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.xlim([20000,160000])
#plt.ylim([0,20])
plt.xlabel('Energy (pes)')
plt.ylabel('Counts/bin')

In [ ]:
count = 0
for hitc in hitc_evt:
    print("*** Peak number {} ***".format(count))
    count += 1
    for slc in hitc:
        for hit in slc:
            print("hit at ({0},{1},{2}; E = {3})".format(hit.X,hit.Y,hit.Z,hit.E)) 

count = 0
for cath_energy_c in cath_energy_evt:
    print("*** Peak number {} ***".format(count))
    count += 1
    sl = 0
    e_peak = 0
    for e in cath_energy_c:
        print("Energy of slice {} = {}".format(sl, e))
        sl += 1
        e_peak += e
    print("Sum check = {}".format(e_peak))

In [ ]:
e_spec = []; ecorr_spec = []; emc_spec = []; r_spec = []; z_spec = []
for ee in range(len(hitc_evt)):
    evt_E = sum([hh.E for hh in hitc_evt[ee]])
    evt_X = sum([hh.X*hh.E for hh in hitc_evt[ee]])
    evt_Y = sum([hh.Y*hh.E for hh in hitc_evt[ee]])
    evt_Z = sum([hh.Z*hh.E for hh in hitc_evt[ee]])
    if(len(hitc_evt[ee]) > 0):
        evt_X /= evt_E
        evt_Y /= evt_E
        evt_Z /= evt_E
    evt_R = np.sqrt(evt_X**2 + evt_Y**2)
    evt_Emc = 0
    if(ee < len(hitc_mc)): evt_Emc = sum([hh.E for hh in hitc_mc[ee]])
    hr = np.array([np.sqrt(hh.X**2 + hh.Y**2) < 120. for hh in hitc_evt[ee]])
    if(evt_E > 0): #hr.all()):
        e_spec.append(evt_E)
        r_spec.append(evt_R)
    emc_spec.append(evt_Emc)
    z_spec.append(evt_Z)

e_spec = np.array(e_spec)

fig = plt.figure(1)
fig.set_figheight(5.0)
fig.set_figwidth(15.0)

ax1 = fig.add_subplot(121);
plt.hist(e_spec,bins=40,label='Corrected energy')
lnd = plt.legend(loc=1)
#plt.scatter(z_spec,e_spec,marker='.')
#plt.ylim([100000,900000])
#plt.xlim([20000,160000])
plt.xlabel('Energy (Q)')
plt.ylabel('Counts/bin')

ax2 = fig.add_subplot(122);
plt.scatter(r_spec,e_spec,alpha=0.5,marker='.')
lnd = plt.legend(loc=1)
plt.xlabel('r (mm)')
plt.ylabel('Energy (Q)')


In [ ]:
array[8,5,3]

In [ ]:
a.argmin()

In [ ]:
abs(a)

In [ ]:
b=a-3

In [ ]:
b

In [ ]:
abs(b)

In [ ]:
a.argmin()

In [ ]:
np.argmin(abs(a_x-3))

In [ ]:
a_x[450]